In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from gensim import corpora
from gensim import models

In [2]:
path='/Volumes/GoogleDrive/내 드라이브/2021-2/커텍마/과제/Final Project/FINAL/'
csv='.csv'

company_list=['당근마켓','라인플러스','롯데정보통신','무신사','비바리퍼블리카','삼성SDS','신세계아이앤씨',
              '우아한형제들','카카오','크래프톤','티맥스소프트','포스코ICT','하이퍼커넥트','한국IBM','한글과컴퓨터',
             'AhnLab','COUPANG','LGCNS','NAVER','NCSOFT']

review_list=[]
career_list=[]
life_list=[]
money_list=[]
culture_list=[]
manager_list=[]
overall_list=[]
sentiment_list=[]

for i in range(len(company_list)):
    data=pd.read_csv(path+company_list[i]+csv, header=0, index_col=0)
    data['overall']=np.mean(data,axis=1)
    review_list.append(list(data.review))
    career_list.append(list(data.career))
    life_list.append(list(data.life))
    money_list.append(list(data.money))
    culture_list.append(list(data.culture))
    manager_list.append(list(data.manager))
    overall_list.append(list(data.overall))

for i in range(len(company_list)):
    for j in range(len(overall_list[i])):
        if overall_list[i][j]>3.5:
            sentiment_list.append('positive')
        elif overall_list[i][j]<2.5:
            sentiment_list.append('negative')
        else: sentiment_list.append('neutral')

In [3]:
import konlpy.tag
import re

In [4]:
for i in range(len(company_list)):
    for j in range(len(review_list[i])):
        review_list[i][j]=re.sub(r'[^\s\w\d]',' ', review_list[i][j])

In [5]:
twitter=konlpy.tag.Twitter()
nva_word=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
dummy_list=[]

for i in range(len(company_list)):
    for j in range(len(review_list[i])):
        twitter_morphs=twitter.pos(review_list[i][j])
        for word, pos in twitter_morphs:
            if pos == 'Noun' or pos == 'Verb' or pos == 'Adjective':
                dummy_list.append(word)
        nva_word[i].append(dummy_list)
        dummy_list=[]

/Users/ijaehwan/opt/anaconda3/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [45]:
unique_nva=[]
for i in range(len(nva_word)):
    for j in range(len(nva_word[i])):
        unique_nva.append(list(set(nva_word[i][j])))
    
uni_nva=[]
for i in range(len(unique_nva)):
    for j in range(len(unique_nva[i])):
        uni_nva.append(unique_nva[i][j])
        
uni_nva=set(uni_nva)

stopword=['회사','라벨','있는','하는', '게임']

#전처리
for i in range(len(nva_word)):
    for j in range(len(nva_word[i])):
        for word in uni_nva:
            if len(word)<2:
                while word in nva_word[i][j]: nva_word[i][j].remove(word)
            if word in stopword:
                while word in nva_word[i][j]: nva_word[i][j].remove(word)

In [46]:
all_nva=[]
all_career=[]
all_life=[]
all_money=[]
all_culture=[]
all_manager=[]
all_overall=[]
for i in range(len(nva_word)):
    all_nva=all_nva+nva_word[i]
    all_career=all_career+career_list[i]
    all_life=all_life+life_list[i]
    all_money=all_money+money_list[i]
    all_culture=all_culture+culture_list[i]
    all_manager=all_manager+manager_list[i]
    all_overall=all_overall+overall_list[i]
    
dataframe_list=[all_nva, all_career, all_life, all_money, all_culture, all_manager, all_overall, sentiment_list]
df=pd.DataFrame(dataframe_list)
df=df.T
df.columns=['NounVerbAdj','career','life','money','culture','manager','overall','sentiment']
df

,NounVerbAdj,career,life,money,culture,manager,overall,sentiment
0,"[밸류, 인정, 받고, 방황]",3,3,3,3,2,2.8,neutral
1,"[들어온지, 마안, 됐지만, 좋아, 보여요]",5,5,5,5,5,5.0,positive
2,"[열정, 실력, 갖추는, 조직]",5,5,5,5,5,5.0,positive
3,"[로켓, 성장하는]",5,5,5,5,5,5.0,positive
4,"[워워벨, 보장, 되는]",4,3,5,5,3,4.0,positive
...,...,...,...,...,...,...,...,...
12295,[다닐만],3,3,4,3,3,3.2,neutral
12296,[팀바팀],4,4,4,4,3,3.8,positive
12297,"[개임, 특성, 단점, 빼면, 좋음]",3,5,5,4,4,4.2,positive
12298,"[복지, 좋음]",2,3,4,2,1,2.4,negative


In [59]:
df.to_csv('Preprocessed Dataframe.csv', index=False, sep = chr(int("15")))


#불러올 때도 구분자를 꼭 기억하자 chr(int("15"))

In [ ]:
#여기까지 전처리#

In [47]:
def build_doc_word_matrix(docs):
    dictionary = corpora.Dictionary(docs)
    corpus = []
    for doc in docs:
        bow = dictionary.doc2bow(doc)
        corpus.append(bow)

    return corpus, dictionary

corpus, dictionary = build_doc_word_matrix(df.iloc[:,0])

In [48]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 6114
Number of documents: 12300


In [56]:
def print_topic_words(model):
    f = open('topic_results_test.txt','w')
    for topic_id in range(model.num_topics):
        word_probs = model.show_topic(topic_id, 2)
        print("Topic ID: {}".format(topic_id))
        f.write(str(topic_id)+'\n')
        for word, prob in word_probs:
            print("\t{}\t{}".format(word, prob))
            f.write(str(word)+'\t'+str(prob)+'\n')
        print("\n")
    f.close()

In [57]:
lda_model = models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary)

In [64]:
#토픽모델링 (1)

In [58]:
print_topic_words(lda_model)

Topic ID: 0
	복지	0.11655901372432709
	보장	0.07036088407039642


Topic ID: 1
	좋은	0.1800929307937622
	최고	0.04122032970190048


Topic ID: 2
	괜찮은	0.02815878950059414
	직원	0.023537902161478996


Topic ID: 3
	사람	0.03181305155158043
	문화	0.03113090619444847


Topic ID: 4
	좋음	0.03632063418626785
	안정	0.020881904289126396




In [60]:
lda_model_2 = models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)

In [61]:
def print_topic_words_2(model):
    f = open('topic_results_test.txt','w')
    for topic_id in range(model.num_topics):
        word_probs = model.show_topic(topic_id, 2)
        print("Topic ID: {}".format(topic_id))
        f.write(str(topic_id)+'\n')
        for word, prob in word_probs:
            print("\t{}\t{}".format(word, prob))
            f.write(str(word)+'\t'+str(prob)+'\n')
        print("\n")
    f.close()

In [ ]:
#토픽모델링 (2)

In [62]:
print_topic_words_2(lda_model_2)

Topic ID: 0
	좋은	0.06945085525512695
	복지	0.043710656464099884


Topic ID: 1
	보장	0.03151746839284897
	되는	0.027433065697550774




In [16]:
len(df[df['sentiment']=='positive'])

4614

In [17]:
len(df[df['sentiment']=='negative'])

3515

In [18]:
len(df[df['sentiment']=='neutral'])

4171